<a href="https://colab.research.google.com/github/mohammadreza-mohammadi94/NLP-Projects/blob/main/IMDB%20Reviews%20-%20BPE%20From%20Scratch/Byte_Pair_Encoding_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
from collections import defaultdict, Counter
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
nltk.download("punkt_tab")

import pandas as pd


# Download Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

# Load Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')  # مسیر فایل رو درست کن
print(f"Number Of Reviews: {len(df)}")

In [ ]:
df.head(5)

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", "", text)
    words = text.split()
    return words

corpus = []
for review in df['review']:
    words = preprocess_text(review)
    corpus.extend(words)

print(f"Number Of Words: {len(corpus)}")
print(f"Few Samples: {corpus[:10]}")

# Functions

In [ ]:
def get_stats(vocab):
    """
    Compute frequency of adjacent symbol pairs in a given vocabulary.

    Args:
        vocab (dict): A dictionary where keys are space-separated strings (words or subwords)
                      and values are their corresponding frequencies.

    Returns:
        dict: A dictionary with symbol pairs as keys (tuples of two strings)
              and their total frequency of co-occurrence as values.
    """
    pairs = defaultdict(int)  # Store frequency of each adjacent symbol pair

    for word, freq in vocab.items():
        symbols = word.split()  # Split the word into individual symbols (tokens/subwords)
        for i in range(len(symbols) - 1):
            pair = (symbols[i], symbols[i + 1])  # Get current symbol pair
            pairs[pair] += freq  # Increment frequency by word's occurrence count

    return pairs


def merge_vocab(pair, vocab):
    """
    Merge the most frequent symbol pair in the vocabulary.

    Args:
        pair (tuple): A tuple of two strings representing the symbol pair to merge.
                      For example, ('l', 'o') to merge into 'lo'.
        vocab (dict): A dictionary where keys are space-separated strings (words/subwords)
                      and values are their frequencies.

    Returns:
        dict: A new vocabulary dictionary with the specified pair merged into a single symbol.
    """
    new_vocab = {}
    bigram = " ".join(pair)       # e.g., ('l', 'o') -> 'l o'
    replacement = "".join(pair)   # e.g., ('l', 'o') -> 'lo'

    for word in vocab:
        # Replace all occurrences of the bigram with the merged symbol
        new_word = word.replace(bigram, replacement)
        new_vocab[new_word] = vocab[word]  # Preserve the frequency

    return new_vocab


def learn_bpe(corpus, num_merges=1000):
    """
    Learn Byte Pair Encoding (BPE) merges from a text corpus.

    Args:
        corpus (list of str): The input corpus where each item is a word or sentence.
        num_merges (int): The maximum number of BPE merge operations to perform.

    Returns:
        tuple:
            - vocab (dict): Final vocabulary after BPE merges, with words represented as space-separated subwords.
            - merges (list of tuple): List of merged symbol pairs in the order they were applied.
    """
    vocab = Counter()

    # Initialize the vocabulary: each word is split into characters + </w> to mark end of word
    for word in corpus:
        word = " ".join(list(word)) + " </w>"
        vocab[word] += 1

    merges = []

    # Perform BPE merges up to num_merges times
    for i in range(num_merges):
        pairs = get_stats(vocab)  # Count frequency of symbol pairs
        if not pairs:
            break  # No more pairs to merge

        best_pair = max(pairs, key=pairs.get)  # Most frequent pair
        vocab = merge_vocab(best_pair, vocab)  # Merge the best pair in the vocab
        merges.append(best_pair)

        # Log progress every 100 merges
        if (i + 1) % 100 == 0:
            print(f"Merge {i + 1}: {best_pair}")

    return vocab, merges


def apply_bpe(word, merges):
    """
    Applies Byte Pair Encoding (BPE) merges to a given word.

    Args:
        word (str): The input word to encode using learned BPE merges.
        merges (list of tuple): List of symbol pairs (bigrams) in the order they were merged during BPE training.

    Returns:
        list of str: The BPE-encoded word as a list of subword tokens.
    """
    # FIXED TYPO: .joint() → .join()
    # Split word into characters and append </w> to mark end of word
    word = " ".join(list(word)) + " </w>"
    word = word.split()

    # Apply each merge operation in sequence
    for pair in merges:
        bigram = pair[0] + " " + pair[1]             # E.g., 't h'
        replacement = pair[0] + pair[1]               # E.g., 'th'
        word = " ".join(word).replace(bigram, replacement).split()

    return word

# Apply Functions

In [ ]:
num_merges = 1000
vocab, merges = learn_bpe(corpus)

print(f"Vocab Size: {len(vocab)}")
print(f"Merges: {merges[:10]}")

# Tokenization
sample_reviews = df['review'].iloc[:3].tolist()
tokenized_reviews = []

for review in sample_reviews:
    words = preprocess_text(review)
    tokens = []
    for word in words:
        tokens.extend(apply_bpe(word, merges))
    tokenized_reviews.append(tokens)

for i, (review, tokens) in enumerate(zip(sample_reviews, tokenized_reviews)):
    print(f"\Review {i+1}: {review[:100]}...")
    print(f"Tokens: {tokens[:20]}...")
    print(f"Number Of Tokens توکن‌ها: {len(tokens)}")

Merge 100: ('c', 'om')
Merge 200: ('stor', 'y</w>')
Merge 300: ('s', 'pe')
Merge 400: ('i', 've</w>')
Merge 500: ('ma', 'y')
Merge 600: ('z', '</w>')
Merge 700: ('hu', 'man</w>')


In [ ]:
# Analysis of Results
unique_tokens = set()
for word in vocab:
    unique_tokens.update(word.split())
print(f"Unique Tokens: {len(unique_tokens)}")

avg_tokens = sum(len(tokens) for tokens in tokenized_reviews) / len(tokenized_reviews)
print(f"Average Tokens: {avg_tokens:.2f}")

with open('bpe_merges.txt', 'w') as f:
    for pair in merges:
        f.write(f"{pair[0]} {pair[1]}\n")
print("Rules saved...")

# مقایسه با NLTK
nltk_tokens = [word_tokenize(review.lower()) for review in sample_reviews]
for i, (bpe_tokens, nltk_tokens) in enumerate(zip(tokenized_reviews, nltk_tokens)):
    print(f"\nReview {i+1}:")
    print(f"BPE Tokens Count: {len(bpe_tokens)}")
    print(f"NLTK Tokens Count: {len(nltk_tokens)}")
    print(f"Few BPE: {bpe_tokens[:10]}...")
    print(f"Few NLTK: {nltk_tokens[:10]}...")